# SANS2D: I(Q) for sample and background

In this notebook, we will be reducing a sample and a background measurements to a one-dimensional $I(Q)$.

It assumes the detector data has been recorded in event mode, while the monitor data has been histogrammed.

The data used in this notebook has been published in [Manasi et al. (2021)](#manasi2021),
and we kindly thank the authors for allowing us to use their data.

**Outline:**

- We will begin by loading the data files containing the sample, direct, and background measurements.
- We will then apply some corrections to beamline components specific to the SANS2D beamline.
- This will be followed by some masking of some saturated or defect detector pixels
- Finally, the sample and background measurement will be converted to the $Q$ dimension

In [1]:
import matplotlib.pyplot as plt
import scipp as sc
from ess import loki, sans
import scippneutron as scn

## Define reduction workflow parameters

We define here whether to include the effects of gravity,
as well as common time-of-flight, wavelength and $Q$ bins for all the measurements.

We also define a range of wavelengths for the monitors that are considered to not be part of the background.

In [2]:
# Include effects of gravity?
gravity = True

tof_bins = sc.linspace(dim='tof', start=0, stop=100000, num=2, unit='us')

# Wavelength binning
wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=281, unit='angstrom')

# Q binning
q_bins = sc.linspace(dim='Q', start=0.05, stop=0.6, num=201, unit='1/angstrom')

# Define the range of wavelengths for the monitors that are considered
# to not be part of the background
monitor_non_background_range = sc.array(dims=['wavelength'],
                                        values=[0.7, 17.1], unit='angstrom')

## Loading data files

We load the following files:

- The direct beam function for the main detector (gives detector efficiency as a function of wavelength)
- The sample measurement
- The direct measurement: this is the run with the empty sample holder/cuvette
- the background measurement: this is the run with only the solvent which the sample is placed in

In [3]:
ds = sc.Dataset()

#Using only one-forth of the full spectra 245760 (reserved for first detector)
spectrum_size =  245760//4

direct_beam = loki.io.load_rkh_wav(
    loki.data.get_path('DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'))

ds['sample'] = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063114.nxs'),
                                   spectrum_size=spectrum_size, tof_bins=tof_bins)

ds['direct'] = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063091.nxs'),
                                   spectrum_size=spectrum_size, tof_bins=tof_bins)

ds['background'] = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063159.nxs'),
                                       spectrum_size=spectrum_size, tof_bins=tof_bins)
ds

FrameworkManager-[Notice] Welcome to Mantid 6.3.0
FrameworkManager-[Notice] Please cite: http://dx.doi.org/10.1016/j.nima.2014.07.029 and this release: http://dx.doi.org/10.5286/Software/Mantid6.3
DownloadInstrument-[Notice] All instrument definitions up to date
LoadRKH-[Notice] LoadRKH started
LoadRKH-[Notice] LoadRKH successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
Load-[Notice] Load started
Load-[Notice] Load successful, Duration 5.27 seconds


Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'period_change_log' has unrecognised units: 'period_number'
Workspace run log 'raw_frames' has unrecognised units: 'frames'
Workspace run log 'veto_log' has unrecognised units: 'is_vetoing'
Workspace run log 'events_log' has unrecognised units: 'events'
Workspace run log 'frame_log' has unrecognised units: 'frame_number'
Workspace run log 'good_frame_log' has unrecognised units: 'is_good'
Workspace run log 'period_log' has unrecognised units: 'period_number'
Workspace run log 'raw_events_log' has unrecognised units: 'events'


ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] 

Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'period_change_log' has unrecognised units: 'period_number'
Workspace run log 'raw_frames' has unrecognised units: 'frames'
Workspace run log 'veto_log' has unrecognised units: 'is_vetoing'
Workspace run log 'events_log' has unrecognised units: 'events'
Workspace run log 'frame_log' has unrecognised units: 'frame_number'
Workspace run log 'good_frame_log' has unrecognised units: 'is_good'
Workspace run log 'period_log' has unrecognised units: 'period_number'
Workspace run log 'raw_events_log' has unrecognised units: 'events'


ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] 

Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'period_change_log' has unrecognised units: 'period_number'
Workspace run log 'raw_frames' has unrecognised units: 'frames'
Workspace run log 'veto_log' has unrecognised units: 'is_vetoing'
Workspace run log 'events_log' has unrecognised units: 'events'
Workspace run log 'frame_log' has unrecognised units: 'frame_number'
Workspace run log 'good_frame_log' has unrecognised units: 'is_good'
Workspace run log 'period_log' has unrecognised units: 'period_number'
Workspace run log 'raw_events_log' has unrecognised units: 'events'


ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] 

<scipp.Dataset>
Dimensions: Sizes[spectrum:61440, tof:1, ]
Coordinates:
  position                  vector3              [m]  (spectrum)  [(-0.5192, -0.48195, 23.281), (-0.517162, -0.48195, 23.281), ..., (0.517162, 0.48195, 23.281), (0.5192, 0.48195, 23.281)]
  sample_position           vector3              [m]  ()  [(0, 0, 19.281)]
  source_position           vector3              [m]  ()  [(0, 0, 0)]
  spectrum                    int32           [None]  (spectrum)  [9, 10, ..., 61447, 61448]
  tof                       float64            [µs]  (tof [bin-edge])  [0, 100000]
Data:
  background              DataArrayView           [None]  (spectrum, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, ..., <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

]
    Attributes:
        Changer                 DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:2, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-14T05:33:15.000000000, 2020-02-14T05:33:34.000000000]
Data:
                             string  [dimensionless]  (time)  ["ET", "FT"]

]
        Det_Temp_FLB            DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:4, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-14T05:33:15.000000000, 2020-02-14T05:33:34.000000000, 2020-02-14T05:55:38.000000000, 2020-02-14T06:08:42.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.1, 17, 16.9, 16.9]

]
        Det_Temp_FLT            DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-14T05:33:15.000000000, 2020-02-14T05:33:34.000000000, 2020-02-14T06:08:42.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.6, 17.6, 17.6]

]
        Det_Temp_FRB            DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-14T05:33:15.000000000, 2020-02-14T05:33:34.000000000, 2020-02-14T06:08:42.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.6, 17.6, 17.6]

]
        Det_Temp_FRT            DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-14T05:33:15.000000000, 2020-02-14T05:33:34.000000000, 2020-02-14T06:08:42.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.6, 17.6, 17.6]

]
        Det_Temp_RLB            DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-14T05:33:15.000000000, 2020-02-14T05:33:34.000000000, 2020-02-14T06:08:42.000000000]
Data:
                            float64  [dimensionless]  (time)  [18, 18, 18]

]
        Det_Te

## Apply corrections to pixel positions

We apply some corrections to the detector pixel and monitor positions,
as the geometry stored in the file is inaccurate.

In [4]:
# Custom SANS2D position offsets
sample_pos_z_offset = 0.053 * sc.units.m
bench_pos_y_offset = 0.001 * sc.units.m
# There is some uncertainity here
monitor4_pos_z_offset = -6.719 * sc.units.m

# Geometry transformation based on the found beam center position 
x_offset = -0.1057 * sc.units.m
y_offset = 0.082735 * sc.units.m

In [5]:
#Inputs for solid angle calculations
ds.coords["pixel_width"] = 0.0035 * sc.units.m
ds.coords["pixel_height"] = 0.002033984375 * sc.units.m

# Change sample position
ds.coords["sample_position"].fields.z += sample_pos_z_offset
# Apply bench offset to pixel positions
ds.coords["position"].fields.y += bench_pos_y_offset

for key in ds:
    ds[key].attrs["monitor4"].value.coords["position"].fields.z += monitor4_pos_z_offset

# Now shift pixels positions to get the correct beam center
ds.coords['position'].fields.x += x_offset
ds.coords['position'].fields.y += y_offset

## Masking

The next step is to mask noisy and saturated pixels,
as well as a time-of-flight range that contains spurious artifacts from the beamline components.

**Note:** We use programatic masks here and not those stored in xml files.

### Mask bad pixels

We mask the edges of the detector, which are usually noisy.
We also mask the region close to the center of the beam,
so as to not include saturated pixels in our data reduction.

In [6]:
mask_edges = (
    (sc.abs(ds.coords['position'].fields.x - x_offset) > sc.scalar(0.48, unit='m')) |
    (sc.abs(ds.coords['position'].fields.y - y_offset) > sc.scalar(0.45, unit='m')))

mask_center = sc.sqrt(
    ds.coords['position'].fields.x**2 +
    ds.coords['position'].fields.y**2) < sc.scalar(0.04, unit='m')

for key in ds:
    ds[key].masks['edges'] = mask_edges
    ds[key].masks['center'] = mask_center

We can inspect that the coordinate corrections and masking were applied correctly by opening the instrument view.

In [7]:
scn.instrument_view(ds['sample'], pixel_size=0.0075)

### Mask Bragg peaks in time-of-flight

We will now take out the time regions with Bragg peaks from the beam stop and detector window,
although in reality the peaks appear only close to the beam stop,
and will make little difference to $I(Q)$.

This could be implemented as masking specific time bins for a specific region in space,
but for now we keep it simple.

In [8]:
mask_tof_min = sc.scalar(13000.0, unit='us')
mask_tof_max = sc.scalar(15750.0, unit='us')
tof_masked_region = sc.concat([ds.coords['tof']['tof', 0],
                               mask_tof_min, mask_tof_max,
                               ds.coords['tof']['tof', -1]], dim='tof')

binned = sc.Dataset()
for key in ds:
    binned[key] = sc.bin(ds[key], edges=[tof_masked_region])
    binned[key].masks['bragg_peaks'] = sc.array(dims=['tof'], values=[False, True, False])
binned

<scipp.Dataset>
Dimensions: Sizes[spectrum:61440, tof:3, ]
Coordinates:
  pixel_height              float64              [m]  ()  [0.00203398]
  pixel_width               float64              [m]  ()  [0.0035]
  position                  vector3              [m]  (spectrum)  [(-0.6249, -0.398215, 23.281), (-0.622862, -0.398215, 23.281), ..., (0.411462, 0.565685, 23.281), (0.4135, 0.565685, 23.281)]
  sample_position           vector3              [m]  ()  [(0, 0, 19.334)]
  source_position           vector3              [m]  ()  [(0, 0, 0)]
  spectrum                    int32           [None]  (spectrum)  [9, 10, ..., 61447, 61448]
  tof                       float64            [µs]  (tof [bin-edge])  [0, 13000, 15750, 100000]
Data:
  background              DataArrayView           [None]  (spectrum, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, ..., <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

]
    Masks:
        bragg_peaks                  bool           [None]  (tof)  [False, True, False]
        center                       bool           [None]  (spectrum)  [False, False, ..., False, False]
        edges                        bool           [None]  (spectrum)  [True, True, ..., True, True]
    Attributes:
        Changer                 DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:2, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-14T05:33:15.000000000, 2020-02-14T05:33:34.000000000]
Data:
                             string  [dimensionless]  (time)  ["ET", "FT"]

]
        Det_Temp_FLB            DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:4, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-14T05:33:15.000000000, 2020-02-14T05:33:34.000000000, 2020-02-14T05:55:38.000000000, 2020-02-14T06:08:42.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.1, 17, 16.9, 16.9]

]
        Det_Temp_FLT            DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-14T05:33:15.000000000, 2020-02-14T05:33:34.000000000, 2020-02-14T06:08:42.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.6, 17.6, 17.6]

]
        Det_Temp_FRB            DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-14T05:33:15.000000000, 2020-02-14T05:33:34.000000000, 2020-02-14T06:08:42.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.6, 17.6, 17.6]

]
        Det_Temp_FRT            DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-14T05:33:15.000000000, 2020-02-14T05:33:34.000000000, 2020-02-14T06:08:42.000000000]
Data:


## Use to_I_of_Q workflow

We now reduce the sample and the background measurements to `Q` using the `sans.to_I_of_Q` workflow.

In that process,
the intensity as a function of `Q` is normalized using the direct measurement and direct beam function.

The workflow needs monitor data from the sample, background, and direct runs to compute the normalization.
It accepts those in the form of a dictionaries:

In [9]:
sample_monitors = {'incident': binned['sample'].attrs["monitor2"].value,
                   'transmission': binned['sample'].attrs["monitor4"].value}

direct_monitors = {'incident': binned['direct'].attrs["monitor2"].value,
                   'transmission': binned['direct'].attrs["monitor4"].value}

background_monitors = {'incident': binned['background'].attrs["monitor2"].value,
                       'transmission': binned['background'].attrs["monitor4"].value}

We then call the workflow on the sample and direct runs:

In [10]:
sample_q = sans.to_I_of_Q(data=binned['sample'],
    data_monitors=sample_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    monitor_non_background_range=monitor_non_background_range)
sample_q.plot()

An interpolation was performed on the direct_beam function. The variances in the direct_beam function have been dropped.


In [11]:
background_q = sans.to_I_of_Q(data=binned['background'],
    data_monitors=background_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    monitor_non_background_range=monitor_non_background_range)
background_q.plot()

An interpolation was performed on the direct_beam function. The variances in the direct_beam function have been dropped.


We are now in a position to subtract the background from the sample measurement:

In [12]:
result = sample_q.bins.sum() - background_q.bins.sum()
result

<scipp.DataArray>
Dimensions: Sizes[Q:200, ]
Coordinates:
  Q                         float64           [1/Å]  (Q [bin-edge])  [0.05, 0.05275, ..., 0.59725, 0.6]
  pixel_height              float64              [m]  ()  [0.00203398]
  pixel_width               float64              [m]  ()  [0.0035]
Data:
                            float32  [dimensionless]  (Q)  [271.075, 290.094, ..., 1.20087, 1.77397]  [0.54415, 0.555004, ..., 3.36386, 1.57349]
Attributes:
  L1                        float64              [m]  ()  [19.334]
  current_period              int64  [dimensionless]  ()  [1]
  experiment_identifier      string  [dimensionless]  ()  ["1920676"]
  gravity                   vector3          [m/s^2]  ()  [(0, -9.80665, 0)]
  incident_beam             vector3              [m]  ()  [(0, 0, 19.334)]
  instrument_name            string           [None]  ()  ["SANS2D"]
  measurement_id             string  [dimensionless]  ()  [""]
  measurement_label          string  [dimensionless]  ()  [""]
  measurement_subid          string  [dimensionless]  ()  [""]
  measurement_type           string  [dimensionless]  ()  [""]
  nperiods                    int64  [dimensionless]  ()  [1]
  placeholder monitor_5   DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[tof:481, ]
Coordinates:
  position                  vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 0)]
  tof                       float64            [µs]  (tof [bin-edge])  [5.5, 50, ..., 99750, 100005]
Data:
                            float64         [counts]  (tof)  [0, 0, ..., 0, 0]  [0, 0, ..., 0, 0]
Attributes:
  instrument_name            string           [None]  ()  ["SANS2D"]
  sample_position           vector3              [m]  ()  [(0, 0, 19.281)]
  spectrum                    int32           [None]  ()  [5]

]
  placeholder monitor_6   DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[tof:481, ]
Coordinates:
  position                  vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 0)]
  tof                       float64            [µs]  (tof [bin-edge])  [5.5, 50, ..., 99750, 100005]
Data:
                            float64         [counts]  (tof)  [0, 0, ..., 0, 0]  [0, 0, ..., 0, 0]
Attributes:
  instrument_name            string           [None]  ()  ["SANS2D"]
  sample_position           vector3              [m]  ()  [(0, 0, 19.281)]
  spectrum                    int32           [None]  ()  [6]

]
  placeholder monitor_7   DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[tof:481, ]
Coordinates:
  position                  vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 0)]
  tof                       float64            [µs]  (tof [bin-edge])  [5.5, 50, ..., 99750, 100005]
Data:
                            float64         [counts]  (tof)  [0, 0, ..., 0, 0]  [0, 0, ..., 0, 0]
Attributes:
  instrument_name            string           [None]  ()  ["SANS2D"]
  sample_position           vector3              [m]  ()  [(0, 0, 19.281)]
  spectrum                    int32           [None]  ()  [7]

]
  placeholder monitor_8   DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[tof:481, ]
Coordinates:
  position                  vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 0)]
  tof                       float64            [µs]  (tof [bin-edge])  [5.5, 50, ..., 99750, 100005]
Data:
                            float64         [counts]  (tof)  [0, 0, ..., 0, 0]  [0, 0, ..., 0, 0]
Attributes:
  instrument_name            string           [None]  ()  ["SANS2D"]
  sample_position           vector3              [m]  ()  [(0, 0, 19.281)]
  spectrum                    int32           [None]  ()  [8]

]
  sample                   PyObject           [None]  ()  [<man

In [13]:
sc.plot(result)

## Wavelength bands

It is often useful to process the data in a small number (~10) of separate wavelength bands.

This can be achieved by requesting 10 bands from the `to_I_of_Q` workflow via the `wavelength_bands` argument.

In [16]:
wavelength_bands = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=11,
                               unit='angstrom')

sample_slices = sans.to_I_of_Q(data=binned['sample'],
    data_monitors=sample_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_bands=wavelength_bands,
    monitor_non_background_range=monitor_non_background_range)

background_slices = sans.to_I_of_Q(data=binned['background'],
    data_monitors=background_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_bands=wavelength_bands,
    monitor_non_background_range=monitor_non_background_range)

result_slices = sample_slices.bins.sum() - background_slices.bins.sum()
result_slices

An interpolation was performed on the direct_beam function. The variances in the direct_beam function have been dropped.
An interpolation was performed on the direct_beam function. The variances in the direct_beam function have been dropped.


<scipp.DataArray>
Dimensions: Sizes[Q:200, wavelength:10, ]
Coordinates:
  Q                         float64           [1/Å]  (Q [bin-edge])  [0.05, 0.05275, ..., 0.59725, 0.6]
  pixel_height              float64              [m]  ()  [0.00203398]
  pixel_width               float64              [m]  ()  [0.0035]
  wavelength                float64             [Å]  (wavelength [bin-edge])  [2, 3.4, ..., 14.6, 16]
Data:
                            float32  [dimensionless]  (Q, wavelength)  [213.651, 282.797, ..., 0, 0]  [2.43766, 2.33443, ..., 0, 0]
Attributes:
  L1                        float64              [m]  ()  [19.334]
  current_period              int64  [dimensionless]  ()  [1]
  experiment_identifier      string  [dimensionless]  ()  ["1920676"]
  gravity                   vector3          [m/s^2]  ()  [(0, -9.80665, 0)]
  incident_beam             vector3              [m]  ()  [(0, 0, 19.334)]
  instrument_name            string           [None]  ()  ["SANS2D"]
  measurement_id             string  [dimensionless]  ()  [""]
  measurement_label          string  [dimensionless]  ()  [""]
  measurement_subid          string  [dimensionless]  ()  [""]
  measurement_type           string  [dimensionless]  ()  [""]
  nperiods                    int64  [dimensionless]  ()  [1]
  placeholder monitor_5   DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[tof:481, ]
Coordinates:
  position                  vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 0)]
  tof                       float64            [µs]  (tof [bin-edge])  [5.5, 50, ..., 99750, 100005]
Data:
                            float64         [counts]  (tof)  [0, 0, ..., 0, 0]  [0, 0, ..., 0, 0]
Attributes:
  instrument_name            string           [None]  ()  ["SANS2D"]
  sample_position           vector3              [m]  ()  [(0, 0, 19.281)]
  spectrum                    int32           [None]  ()  [5]

]
  placeholder monitor_6   DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[tof:481, ]
Coordinates:
  position                  vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 0)]
  tof                       float64            [µs]  (tof [bin-edge])  [5.5, 50, ..., 99750, 100005]
Data:
                            float64         [counts]  (tof)  [0, 0, ..., 0, 0]  [0, 0, ..., 0, 0]
Attributes:
  instrument_name            string           [None]  ()  ["SANS2D"]
  sample_position           vector3              [m]  ()  [(0, 0, 19.281)]
  spectrum                    int32           [None]  ()  [6]

]
  placeholder monitor_7   DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[tof:481, ]
Coordinates:
  position                  vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 0)]
  tof                       float64            [µs]  (tof [bin-edge])  [5.5, 50, ..., 99750, 100005]
Data:
                            float64         [counts]  (tof)  [0, 0, ..., 0, 0]  [0, 0, ..., 0, 0]
Attributes:
  instrument_name            string           [None]  ()  ["SANS2D"]
  sample_position           vector3              [m]  ()  [(0, 0, 19.281)]
  spectrum                    int32           [None]  ()  [7]

]
  placeholder monitor_8   DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[tof:481, ]
Coordinates:
  position                  vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 0)]
  tof                       float64            [µs]  (tof [bin-edge])  [5.5, 50, ..., 99750, 100005]
Data:
                            float64         [counts]  (tof)  [0, 0, ..., 0, 0]  [0, 0, ..., 0, 0]
Attributes:
  instrument_name            string           [None]  ()  ["SANS2D"]
  sample_position           vector3              [m]  ()  [(0, 0, 19.281)]
  spectrum              

In [15]:
collapsed = sc.collapse(result_slices, keep='Q')
sc.plot(collapsed)

### Comparison with Mantid

Here we are comparing with data reduction results obtained using Mantid Workbench (version 6.3.0). 

In [19]:
def load_rkh_q(filename):
    return scn.load(
           filename=filename,
           mantid_alg='LoadRKH',
           mantid_args={'FirstColumnValue':'MomentumTransfer'})

In [22]:
mantid_q1d_file = '63114_rear_1D_2.0_16.0.txt'
mantid_q1d = load_rkh_q(filename=mantid_q1d_file)
#Scaling (normalization) factor 
sc.plot({'scipp':  result, 'mantid': mantid_q1d})

LoadRKH-[Notice] LoadRKH started
LoadRKH-[Notice] LoadRKH successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds


In [21]:
mantid_wavelength_slices = ['63114_rear_1D_2.0_3.0.txt', '63114_rear_1D_3.0_4.0.txt',
                           '63114_rear_1D_4.0_5.0.txt', '63114_rear_1D_5.0_6.0.txt',
                           '63114_rear_1D_6.0_7.0.txt', '63114_rear_1D_7.0_8.0.txt',
                           '63114_rear_1D_9.0_10.0.txt', '63114_rear_1D_10.0_11.0.txt',
                           '63114_rear_1D_11.0_12.0.txt', '63114_rear_1D_12.0_13.0.txt',
                           '63114_rear_1D_13.0_14.0.txt', '63114_rear_1D_14.0_15.0.txt',
                           '63114_rear_1D_15.0_16.0.txt']
wav_index = 1
mantid_wav_slice = load_rkh_q(filename=mantid_wavelength_slices[wav_index])
sc.plot({'scipp':collapsed[f'wavelength:{wav_index}'], 'mantid':mantid_wav_slice})

LoadRKH-[Notice] LoadRKH started
LoadRKH-[Notice] LoadRKH successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds


## References

<div id="manasi2021"></div>

Manasi I., Andalibi M. R., Atri R. S., Hooton J., King S. M., Edler K. J., **2021**,
*Self-assembly of ionic and non-ionic surfactants in type IV cerium nitrate and urea based deep eutectic solvent*,
[J. Chem. Phys. 155, 084902](https://doi.org/10.1063/5.0059238)